In [1]:
import os 
import sys 
sys.path.append(os.path.join("..", ".."))
import cv2
import numpy as np 
from utils.imutils import jimshow as show 
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt

import glob
import pandas as pd

In [ ]:
# sudo apt-get update
# sudo apt-get install -y python3-opencv
# pip install opencv-python matplotlib
# pip install pandas 

Extracting the chosen flower

In [2]:
filepath_f1 = os.path.join("..", "..", "..", "..", "cds-vis-data", "flowers", "image_0001.jpg")
image_f1 = cv2.imread(filepath_f1)
filename_f1 = filepath_f1.split("/")[-1]

In [3]:
hist_f1 = cv2.calcHist([image_f1], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
norm_hist_f1 = cv2.normalize(hist_f1, hist_f1, 0, 1.0, cv2.NORM_MINMAX)

Comparing all other flowers to the extracted flower

In [4]:
filepath_f = os.path.join("..", "..", "..", "..", "cds-vis-data", "flowers")

In [5]:
distances = pd.DataFrame(columns= ("Filename", "Distance"))

for file in sorted(os.listdir(filepath_f)):
    filepath_fx = os.path.join(filepath_f, file)
    image_fx = cv2.imread(filepath_fx)
    filename_fx = file.split(".jpg")[0]
    hist_fx = cv2.calcHist([image_fx], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
    norm_hist_fx = cv2.normalize(hist_fx, hist_fx, 0, 1.0, cv2.NORM_MINMAX)

    dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)
    row_fx = [filename_fx, dist_fx]
        
    if len(distances.index)<6:
        distances.loc[len(distances)] = row_fx
                       
    elif ((distances.where(distances["Distance"]>dist_fx))["Distance"]>0).any():
        to_rep = (distances.where(distances["Distance"]>dist_fx))
        to_rep = (to_rep[to_rep["Distance"] == to_rep["Distance"].max()]).values.flatten().tolist()
        distances.replace(to_rep, row_fx)
        
    else: 
        pass 


outpath = os.path.join("..", "out", "distances.csv")
distances.to_csv(outpath) 

In [6]:
distances_all = pd.DataFrame(columns= ("Filename", "Distance"))

for file in sorted(os.listdir(filepath_f)):
    filepath_fx = os.path.join(filepath_f, file)
    image_fx = cv2.imread(filepath_fx)
    filename_fx = file.split(".jpg")[0]
    hist_fx = cv2.calcHist([image_fx], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
    norm_hist_fx = cv2.normalize(hist_fx, hist_fx, 0, 1.0, cv2.NORM_MINMAX)

    dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)
    row_fx = [filename_fx, dist_fx]

    distances_all.loc[len(distances_all)] = row_fx

else: 
    pass

In [7]:
distances_all.nsmallest(5, ["Distance"])

,Filename,Distance
0,image_0001,0.00000
927,image_0928,178.12428
875,image_0876,188.54842
772,image_0773,190.08066
141,image_0142,190.20916


In [68]:
# Define a function to update the df with new distances
def update_distance_df(filename, distance, df):
    df.loc[len(df.index)] = [filename, distance] # .loc is uded to access rows and columns by label(s)
    return df

In [151]:
tester.loc[len(tester)]

Filename    test
Distance    10.0
Name: 5, dtype: object

In [103]:
distance_df = pd.DataFrame(columns=("Filename", "Distance"))

for file in sorted(os.listdir(filepath_f)):
    filepath_fx = os.path.join(filepath_f, file)
    image_fx = cv2.imread(filepath_fx)
    filename_fx = file.split(".jpg")[0]
    hist_fx = cv2.calcHist([image_fx], channels = [0,1,2], mask = None, histSize = [255,255,255], ranges = [0,256, 0,256,0,256])
    norm_hist_fx = cv2.normalize(hist_fx, hist_fx, 0, 1.0, cv2.NORM_MINMAX)

    dist_fx = round(cv2.compareHist(norm_hist_f1, norm_hist_fx, cv2.HISTCMP_CHISQR),5)

    if len(distance_df) < 7: 
        # If there is less than give images appended to the df, simply append the information
        update_distance_df(filename_fx, dist_fx, distance_df)
    else:
        # Find image with the highest distance value in the df - so the one that are least similar
        # pd .idxmax() function returns the index with the maximum value
        max_dist_idx = distance_df['Distance'].idxmax()

        max_dist = distance_df.loc[max_dist_idx, 'Distance']

        # If the dist of the new image is smaller than the max distance in df, then...
        if dist_fx < max_dist:
            # Drop the row with the max value 
            distance_df = distance_df.drop(index = max_dist_idx)
            # Append the image name and distance value
            update_distance_df(filename_fx, dist_fx, distance_df)
    
print(distance_df)

     Filename      Distance
0  image_0001       0.00000
1  image_0002    6650.67341
3  image_0004     196.45550
4  image_0005    9006.81127
5  image_0006    9252.33121
6  image_1360  215264.63800


In [145]:
tester = distance_df.nsmallest(5, ["Distance"])

In [146]:
tester

,Filename,Distance
0,image_0001,0.00000
3,image_0004,196.45550
1,image_0002,6650.67341
4,image_0005,9006.81127
5,image_0006,9252.33121


In [156]:
row_fx = pd.Series(['testername', 8])
pd.concat([tester, row_fx], axis=1)

,Filename,Distance,0
0,image_0001,0.00000,testername
3,image_0004,196.45550,NaN
1,image_0002,6650.67341,8
4,image_0005,9006.81127,NaN
5,test,10.00000,NaN


In [149]:
dist_fx = 30
filename_fx = "works? "

max_dist_idx = tester['Distance'].idxmax()
print(max_dist_idx)
max_dist = tester.loc[max_dist_idx, 'Distance']
print(max_dist)

if len(tester.index) < 6: 
    update_distance_df("test", 10, tester)
    print(tester)
elif dist_fx < max_dist:
    tester = tester.drop(index = max_dist_idx)
    update_distance_df(filename_fx, dist_fx, tester)
    print(tester)

5
9252.33121
     Filename    Distance
0  image_0001     0.00000
3  image_0004   196.45550
1  image_0002  6650.67341
4  image_0005  9006.81127
5        test    10.00000


In [144]:
len(tester.index)

5

In [100]:
tester.drop(index = max_dist_idx)


,Filename,Distance
0,image_0001,0.00000
663,image_0928,178.12428
627,image_0876,188.54842
553,image_0773,190.08066
5,test,8.00000
6,test,8.00000
7,test,10.00000
8,test,10.00000
9,works?,30.00000
10,works?,30.00000


In [57]:
max_d = tester['Distance'].idxmax()
print(max_d)

102


In [59]:
tester.drop(index = max_d)

,Filename,Distance
0,image_0001,0.00000
663,image_0928,178.12428
627,image_0876,188.54842
553,image_0773,190.08066


In [77]:
update_distance_df("test", 8, tester)

,Filename,Distance
0,image_0001,0.00000
663,image_0928,178.12428
627,image_0876,188.54842
553,image_0773,190.08066
102,image_0142,190.20916
5,test,8.00000
6,test,8.00000
